# Covid NLP Tweet Sentiment Analysis

## We would be using two models

* Bag of Words
    * Multinomial NB
    * PassiveAggressiveClassifer
 

In [1]:
import pandas as pd
import numpy as np

## Bag of Words Model

----------------------------------------

## Train Set

In [2]:
df=pd.read_csv('D:/Data Sets/Corona_NLP text classification/Corona_NLP_train.csv',encoding='latin-1')

In [3]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
#to see the different categories/sentiments
set(df['Sentiment'])

{'Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral', 'Positive'}

In [5]:
#df.iloc[df.shape[0]-1] 

In [6]:
df.shape

(41157, 6)

In [7]:
y=df['Sentiment'].values

In [8]:
#encoding labels
from sklearn.preprocessing import LabelEncoder as le
le=le()
y=le.fit_transform(y)

In [9]:
y

array([3, 4, 4, ..., 4, 3, 2])

In [10]:
tweets=list(df['OriginalTweet'])

In [11]:
tweets[:2]

['@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8',
 'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order']

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer as ps
from sklearn.feature_extraction.text import CountVectorizer as cv

In [14]:
import re
from tqdm import tqdm

ps=ps()
corpus=[]
for i in tqdm(range(len(tweets))):
    tweet=re.sub('[^a-zA-Z]',' ',tweets[i])
    tweet=tweet.lower()
    #print(tweet)
    tweet=tweet.split()
    tweet=[ps.stem(word) for word in tweet if word not in stopwords.words('english')]
    tweet=' '.join(tweet)
    corpus.append(tweet)

100%|███████████████████████████████████████████████████████████████████████████| 41157/41157 [06:50<00:00, 100.26it/s]


In [15]:
cv=cv(max_features=2500)
x=cv.fit_transform(corpus).toarray()

In [16]:
x.shape

(41157, 2500)

In [17]:
c=0
for i in tqdm(x):
    for j in i:
        if j>0:
            c+=1
print(c)

100%|██████████████████████████████████████████████████████████████████████████| 41157/41157 [00:35<00:00, 1143.92it/s]

629912


In [ ]:
#x,y are our variables. This is our train set so we will fit the model on this data 
#and then will read the test file to see how good our model really is

In [18]:
x_train,y_train=x,y

In [ ]:
#we will also be comparing different models

In [43]:
from sklearn.naive_bayes import MultinomialNB as nb
classifier_nb=nb(alpha=1000)
classifier_nb.fit(x_train,y_train)

MultinomialNB(alpha=1000)

In [32]:
from sklearn.linear_model import PassiveAggressiveClassifier as pac
classifier_pac=pac()
classifier_pac.fit(x_train,y_train)

PassiveAggressiveClassifier()

## Test Set

In [21]:
df_test=pd.read_csv('D:/Data Sets/Corona_NLP text classification/Corona_NLP_test.csv',encoding='latin-1')

In [22]:
df_test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [23]:
tweets_test=df_test['OriginalTweet']
y_test=df_test['Sentiment'].values

In [24]:
#encoding the dependent variable
y_test=le.fit_transform(y_test)

In [25]:
y_test

array([0, 4, 1, ..., 3, 0, 1])

In [26]:
from nltk.stem import PorterStemmer as ps

test_corpus=[]
ps_test=ps()
for i in tqdm(range(len(tweets_test))):
    test_tweet=re.sub('[^a-zA-Z]',' ',tweets_test[i])
    test_tweet=test_tweet.lower()
    #print(test_tweet)
    test_tweet=test_tweet.split()
    test_tweet=[ps_test.stem(w) for w in test_tweet if w not in stopwords.words('english')]
    test_tweet=' '.join(test_tweet)
    test_corpus.append(test_tweet)

100%|██████████████████████████████████████████████████████████████████████████████| 3798/3798 [00:43<00:00, 87.13it/s]


In [34]:
from sklearn.feature_extraction.text import CountVectorizer as cv

cv=cv(max_features=2500)
x_test=cv.fit_transform(test_corpus).toarray()

count=0
for i in tqdm(x_test):
    for j in i:
        if j>0:
            count+=1
print(count)

100%|████████████████████████████████████████████████████████████████████████████| 3798/3798 [00:03<00:00, 1222.75it/s]

63427


In [44]:
#predicting values from the test set
y_pred_nb=classifier_nb.predict(x_test)
y_pred_pac=classifier_pac.predict(x_test)

In [36]:
from sklearn.metrics import classification_report,accuracy_score

In [45]:
#Multinomial NaiveBayes
print('Accuracy is:',accuracy_score(y_test,y_pred_nb))
print(classification_report(y_test,y_pred_nb))

Accuracy is: 0.2632964718272775
              precision    recall  f1-score   support

           0       0.41      0.01      0.02       592
           1       0.24      0.01      0.02       599
           2       0.30      0.25      0.28      1041
           3       0.09      0.02      0.03       619
           4       0.26      0.75      0.39       947

    accuracy                           0.26      3798
   macro avg       0.26      0.21      0.15      3798
weighted avg       0.26      0.26      0.18      3798



In [38]:
#PassiveAggressiveClassifier
print('Accuracy is:',accuracy_score(y_test,y_pred_pac))
print(classification_report(y_test,y_pred_pac))

Accuracy is: 0.25223802001053186
              precision    recall  f1-score   support

           0       0.20      0.11      0.14       592
           1       0.18      0.26      0.22       599
           2       0.29      0.40      0.34      1041
           3       0.30      0.22      0.26       619
           4       0.25      0.19      0.22       947

    accuracy                           0.25      3798
   macro avg       0.24      0.24      0.23      3798
weighted avg       0.25      0.25      0.24      3798

